## Setup

In [1]:
GLOBAL_SEED = 42

import os
os.environ['PYTHONHASHSEED'] = str(GLOBAL_SEED)
import sys

import time
import gc
from tqdm import tqdm
import datetime
import pickle
import random as rnd
from glob import glob
import pandas as pd
import numpy as np
from numpy import random as np_rnd
import warnings
from math import ceil
import copy

import seaborn as sns
from matplotlib import pyplot as plt
from matplotlib import rcParams
from itertools import combinations
from collections import Counter

import optuna
from optuna import Trial, create_study
from optuna.samplers import TPESampler

from sklearn import linear_model as lm
from sklearn.model_selection import train_test_split as tts
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder, OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.model_selection import StratifiedKFold

from sklearn.impute import KNNImputer
# explicitly require this experimental feature
from sklearn.experimental import enable_iterative_imputer  # noqa
# now you can import normally from sklearn.impute
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor
from itertools import product

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

from scipy.stats import f_oneway
from scipy.stats import pearsonr
from scipy.special import softmax

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import AdamW
from torch.utils.data import DataLoader, Dataset
from transformers import get_polynomial_decay_schedule_with_warmup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# display setting
warnings.filterwarnings(action='ignore')
rcParams['axes.unicode_minus'] = False

In [2]:
def seed_everything(seed=42):
    os.environ['PYTHONHASHSEED'] = str(seed)
    # python random
    rnd.seed(seed)
    # numpy random
    np_rnd.seed(seed)
    # tf random
    try:
        tf_rnd.set_seed(seed)
    except:
        pass
    # RAPIDS random
    try:
        cupy.random.seed(seed)
    except:
        pass
    # pytorch random
    try:
        torch.manual_seed(seed)
        torch.cuda.manual_seed(seed)
        torch.backends.cudnn.deterministic = True
    except:
        pass

def pickleIO(obj, src, op="w"):
    if op=="w":
        with open(src, op + "b") as f:
            pickle.dump(obj, f)
    elif op=="r":
        with open(src, op + "b") as f:
            tmp = pickle.load(f)
        return tmp
    else:
        print("unknown operation")
        return obj

def findIdx(data_x, col_names):
    return [int(i) for i, j in enumerate(data_x) if j in col_names]

def diff(first, second):
    second = set(second)
    return [item for item in first if item not in second]

def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print('Error: Creating directory. ' + directory)
        
def week_of_month(dt):
    """ 
        Returns the week of the month for the specified date.
    """
    first_day = dt.replace(day=1)
    dom = dt.day
    adjusted_dom = dom + (1 + first_day.weekday()) % 7
    return int(ceil(adjusted_dom/7.0))

def quality_to_class(x, threshold):
    y_pred_class = []
    for i in x:
        tmp_class = len(threshold)
        for k, v in threshold.items():
            if i < v:
                tmp_class = k
                break
        y_pred_class.append(tmp_class)
    return y_pred_class

In [3]:
class CFG:
    debug = False
    TF = True
    product_mapper = {
        "A": ["T010305", "T010306", "T050304", "T050307"],
        "O": ["T100304", "T100306"],
        "T": ["T100304", "T100306"],
    }
    line_mapper = {
        "T010305": "A", "T010306": "A", "T050304": "A", "T050307": "A",
        "T100304": "O_T", "T100306": "O_T",
    }
    line_groups = [
        ["T010305", "T010306"],
        ["T050304", "T050307"],
        ["T100304", "T100306"],
    ]
    classes = [0, 1, 2]
    time_features = ["month", "day", "weekday", "week_of_month", "hour", "office_hour", "sec_in_day", "sin_in_day", "cos_in_day"]
    
    epochs = 20
    batch_size = 4
    early_stopping_rounds = 10
    eta = 5e-4
    weight_decay = 1e-4

In [4]:
quality_threshold = pickleIO(None, "../datasets/dataset_valid/quality_threshold.pkl", "r")
quality_threshold['T010305_T010306_T050304_T050307'] = {0: (0.525046 + 0.525086) / 2, 1: (0.534843 + 0.535279) / 2}

In [5]:
quality_threshold = {
    "T010305_T010306": {0: 0.52507, 1: 0.53490},
    "T050304_T050307": {0: 0.52507, 1: 0.53490},
    "T100304_T100306": {0: 0.52507, 1: 0.53490},
}

In [6]:
quality_threshold

{'T010305_T010306': {0: 0.52507, 1: 0.5349},
 'T050304_T050307': {0: 0.52507, 1: 0.5349},
 'T100304_T100306': {0: 0.52507, 1: 0.5349}}

## Loading Data

In [7]:
# information Provided by Dacon

# PRODUCT_ID : 제품의 고유 ID
# Y_Class : 제품 품질 상태(Target) 
# 0 : 적정 기준 미달 (부적합)
# 1 : 적합
# 2 : 적정 기준 초과 (부적합)
# Y_Quality : 제품 품질 관련 정량적 수치
# TIMESTAMP : 제품이 공정에 들어간 시각
# LINE : 제품이 들어간 공정 LINE 종류 ('T050304', 'T050307', 'T100304', 'T100306', 'T010306', 'T010305' 존재)
# PRODUCT_CODE : 제품의 CODE 번호 ('A_31', 'T_31', 'O_31' 존재)
# X_1 ~ X_2875 : 공정 과정에서 추출되어 비식별화된 변수

In [8]:
df_full = pd.read_csv("../datasets/train.csv")
df_full.columns = df_full.columns.str.lower()

In [9]:
# # time feature engineernig
# df_full["month"] = df_full["timestamp"].dt.month
# df_full["day"] = df_full["timestamp"].dt.day
# df_full["weekday"] = df_full["timestamp"].dt.weekday
# df_full["week_of_month"] = df_full["timestamp"].apply(week_of_month)
# df_full["hour"] = df_full["timestamp"].dt.hour
# df_full["office_hour"] = df_full["hour"].apply(lambda x: 1 if ((x >= 9) & (x < 18)) else 0)
# df_full["sec_in_day"] = (df_full["timestamp"] - df_full["timestamp"].dt.normalize()).dt.total_seconds() / 3600
# df_full["sin_in_day"] = np.sin(2 * np.pi * df_full["sec_in_day"].values)
# df_full["cos_in_day"] = np.cos(2 * np.pi * df_full["sec_in_day"].values)

In [10]:
# df_train = pickleIO(None, "../../datasets/dataset_valid2/df_train.pkl", "r")
# df_valid = pickleIO(None, "../../datasets/dataset_valid2/df_valid.pkl", "r")

In [11]:
# # time feature engineernig
# df_train["month"] = df_train["timestamp"].dt.month
# df_train["day"] = df_train["timestamp"].dt.day
# df_train["weekday"] = df_train["timestamp"].dt.weekday
# df_train["week_of_month"] = df_train["timestamp"].apply(week_of_month)
# df_train["hour"] = df_train["timestamp"].dt.hour
# df_train["office_hour"] = df_train["hour"].apply(lambda x: 1 if ((x >= 9) & (x < 18)) else 0)
# df_train["sec_in_day"] = (df_train["timestamp"] - df_train["timestamp"].dt.normalize()).dt.total_seconds() / 3600
# df_train["sin_in_day"] = np.sin(2 * np.pi * df_train["sec_in_day"].values)
# df_train["cos_in_day"] = np.cos(2 * np.pi * df_train["sec_in_day"].values)

In [12]:
# # time feature engineernig
# df_valid["month"] = df_valid["timestamp"].dt.month
# df_valid["day"] = df_valid["timestamp"].dt.day
# df_valid["weekday"] = df_valid["timestamp"].dt.weekday
# df_valid["week_of_month"] = df_valid["timestamp"].apply(week_of_month)
# df_valid["hour"] = df_valid["timestamp"].dt.hour
# df_valid["office_hour"] = df_valid["hour"].apply(lambda x: 1 if ((x >= 9) & (x < 18)) else 0)
# df_valid["sec_in_day"] = (df_valid["timestamp"] - df_valid["timestamp"].dt.normalize()).dt.total_seconds() / 3600
# df_valid["sin_in_day"] = np.sin(2 * np.pi * df_valid["sec_in_day"].values)
# df_valid["cos_in_day"] = np.cos(2 * np.pi * df_valid["sec_in_day"].values)

In [13]:
# df_train.info()

In [14]:
# df_train.head()

In [15]:
# df_train["y_quality"]

In [16]:
# df_train.groupby(["line", "product_code"]).size()

In [17]:
# df_valid.info()

In [18]:
# df_valid.head()

In [19]:
# df_valid["y_quality"]

In [20]:
# df_valid.groupby(["line", "product_code"]).size()

In [21]:
df_full["tmp"] = df_full["product_code"].apply(lambda x: 1 if x == "A_31" else 0)
display(df_full.groupby(["tmp", "y_class"])["y_quality"].describe().T)
df_full = df_full.drop("tmp", axis=1)

tmp              0                                 1                       
y_class          0           1          2          0           1          2
count    28.000000  289.000000  32.000000  60.000000  118.000000  71.000000
mean      0.521246    0.530272   0.538753   0.520646    0.530209   0.543508
std       0.005765    0.002334   0.004931   0.003990    0.002705   0.008733
min       0.502517    0.525213   0.534951   0.500856    0.525086   0.535279
25%       0.520467    0.528483   0.535541   0.519388    0.527989   0.537330
50%       0.523422    0.530308   0.536237   0.521315    0.530353   0.539235
75%       0.524612    0.532119   0.539517   0.523522    0.532332   0.547506
max       0.525067    0.534837   0.551279   0.525046    0.534843   0.578841

In [22]:
df_full["tmp"] = df_full["product_code"].apply(lambda x: 1 if x == "A_31" else 0)
display(df_full.groupby("tmp")["y_class"].describe().T)
df_full = df_full.drop("tmp", axis=1)

tmp,0,1
count,349.000000,249.000000
mean,1.011461,1.044177
std,0.415069,0.725442
min,0.000000,0.000000
25%,1.000000,1.000000
50%,1.000000,1.000000
75%,1.000000,2.000000
max,2.000000,2.000000


In [23]:
df_full["tmp"] = df_full["product_code"].apply(lambda x: 1 if x == "A_31" else 0)
print(df_full.groupby("tmp")["y_class"].value_counts(normalize=True))
df_full = df_full.drop("tmp", axis=1)

tmp  y_class
0    1          0.828080
     2          0.091691
     0          0.080229
1    1          0.473896
     2          0.285141
     0          0.240964
Name: y_class, dtype: float64


In [24]:
df_test = pd.read_csv("../datasets/test.csv")
df_test.columns = df_test.columns.str.lower()

In [25]:
# # time feature engineernig
# df_test["month"] = df_test["timestamp"].dt.month
# df_test["day"] = df_test["timestamp"].dt.day
# df_test["weekday"] = df_test["timestamp"].dt.weekday
# df_test["week_of_month"] = df_test["timestamp"].apply(week_of_month)
# df_test["hour"] = df_test["timestamp"].dt.hour
# df_test["office_hour"] = df_test["hour"].apply(lambda x: 1 if ((x >= 9) & (x < 18)) else 0)
# df_test["sec_in_day"] = (df_test["timestamp"] - df_test["timestamp"].dt.normalize()).dt.total_seconds() / 3600
# df_test["sin_in_day"] = np.sin(2 * np.pi * df_test["sec_in_day"].values)
# df_test["cos_in_day"] = np.cos(2 * np.pi * df_test["sec_in_day"].values)

In [26]:
df_test.head()

,product_id,timestamp,line,product_code,x_1,x_2,x_3,x_4,x_5,x_6,...,x_2866,x_2867,x_2868,x_2869,x_2870,x_2871,x_2872,x_2873,x_2874,x_2875
0,TEST_000,2022-09-09 2:01,T100306,T_31,2.0,94.0,0.0,45.0,10.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TEST_001,2022-09-09 2:09,T100304,T_31,2.0,93.0,0.0,45.0,11.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,TEST_002,2022-09-09 8:42,T100304,T_31,2.0,95.0,0.0,45.0,11.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,TEST_003,2022-09-09 10:56,T010305,A_31,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,TEST_004,2022-09-09 11:04,T010306,A_31,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Training by Lines

* "T010305", "T010306"
* "T050304", "T050307"
* "T100304", "T100306"

In [27]:
architecture_root_path = "./architectures/"
architecture_name = "daheeRef_bylines_dnn_try1"
architecture_path = architecture_root_path + architecture_name + "/"
createFolder(architecture_path)

In [28]:
# Shuffling
df_full = df_full.sample(frac=1, random_state=GLOBAL_SEED).reset_index(drop=True)

In [29]:
# Label Encoding on Line
lbe = LabelEncoder()
lbe.fit(["T010305", "T010306", "T050304", "T050307", "T100304", "T100306"])

LabelEncoder()

In [30]:
df_meta = {"reg_qual": None, "cls_cls": None, "reg_cls": None}

## Define helper functions

In [31]:
def get_optimizer_params(model, eta, weight_decay):
    no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
    optimizer_parameters = [
        # apply weight decay
        {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
         'lr': eta, 'weight_decay': weight_decay},
        # don't apply weight decay for LayerNormalization layer
        {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
         'lr': eta, 'weight_decay': 0.0},
    ]
    return optimizer_parameters


def get_scheduler(optimizer, num_warmup_steps, num_training_steps):
    scheduler = get_polynomial_decay_schedule_with_warmup(
        optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=num_training_steps, power=0.5, lr_end=1e-7
    )
    return scheduler


class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)

In [32]:
# optuna function
def optuna_objective_function(trial: Trial, fold, train_x, train_y, valid_x, valid_y, output_container):
    
    tuning_params = {
        "dropoutRate": trial.suggest_float("dropoutRate", 0.2, 0.8),
        "hidden_layers": trial.suggest_int("hidden_layers", 2, 10),
        "layer_depth": trial.suggest_int("layer_depth", 2, 2),
        "nblocks": trial.suggest_int("nblocks", 2, 8),
        "batch_norm": trial.suggest_categorical("batch_norm", [True, False]),
    }
    
    seed_everything(fold)
    model, score = do_fold_training(tuning_params)
    optuna_score = score["valid_loss"]
    
    if optuna_score < output_container["optuna_score"]:
        output_container["model"] = model
        output_container["optuna_score"] = optuna_score

    return optuna_score

import operator
class Optuna_EarlyStoppingCallback(object):
    """Early stopping callback for Optuna."""

    def __init__(self, early_stopping_rounds: int, direction: str = "minimize") -> None:
        self.early_stopping_rounds = early_stopping_rounds

        self._iter = 0

        if direction == "minimize":
            self._operator = operator.lt
            self._score = np.inf
        elif direction == "maximize":
            self._operator = operator.gt
            self._score = -np.inf
        else:
            ValueError(f"invalid direction: {direction}")

    def __call__(self, study: optuna.Study, trial: optuna.Trial) -> None:
        """Do early stopping."""
        if self._operator(study.best_value, self._score):
            self._iter = 0
            self._score = study.best_value
        else:
            self._iter += 1

        if self._iter >= self.early_stopping_rounds:
            study.stop()

In [33]:
class CustomDataset(Dataset):
    def __init__(self, feature, label):
        self.feature = feature
        self.label = label
    def __len__(self):
        return len(self.feature)        
    def __getitem__(self, idx):
        return {'feature': self.feature[idx], 'label': self.label[idx]}

class LinearBlock(nn.Module):
    def __init__(self, input_shape, dropoutRate=1/2, hidden_layers=16, layer_depth=2):
        super(LinearBlock, self).__init__()
        self.model = nn.Sequential(
            nn.Dropout(dropoutRate),
            *[nn.Sequential(
                nn.Linear((input_shape if i == 0 else hidden_layers), hidden_layers),
                nn.ReLU()
            ) for i in range(layer_depth)],
        )
    def forward(self, features):
        return self.model(features)

class ResidualLinearBlock(nn.Module):
    def __init__(self, input_shape, dropoutRate=1/2, hidden_layers=16, layer_depth=2, batch_norm=False):
        super(ResidualLinearBlock, self).__init__()
        
        if batch_norm:
            self.pool = nn.Sequential(
                nn.Linear(input_shape, hidden_layers),
                nn.BatchNorm1d(hidden_layers),
            )

            self.resd = nn.Sequential(
                nn.Dropout(dropoutRate),
                nn.Linear(hidden_layers, hidden_layers),
                nn.BatchNorm1d(hidden_layers),
                nn.ReLU(),
                nn.Linear(hidden_layers, hidden_layers),
                nn.BatchNorm1d(hidden_layers),
            )
        else:
            self.pool = nn.Sequential(
                nn.Linear(input_shape, hidden_layers),
            )

            self.resd = nn.Sequential(
                nn.Dropout(dropoutRate),
                nn.Linear(hidden_layers, hidden_layers),
                nn.ReLU(),
                nn.Linear(hidden_layers, hidden_layers),
            )           
        
    def forward(self, features):
        x = self.pool(features)
        return F.relu(self.resd(x) + x)

### Training with tuning parameters - Regression (y_quality) 

In [34]:
class CustomModel(nn.Module):
    def __init__(self, params):
        super(CustomModel, self).__init__()
        self.params = params
        
        self.model = nn.Sequential(
            *[ResidualLinearBlock(
                params["nfeatures"] if i == 0 else max(4, int(params["hidden_layers"] / (2**(i-1)))),
                dropoutRate=params["dropoutRate"],
                hidden_layers=max(4, int(params["hidden_layers"] / (2**i))),
                layer_depth=params["layer_depth"],
                batch_norm=params["batch_norm"]) for i in range(params["nblocks"])
             ],
        )

        self.regressor = nn.Linear(list(self.model.named_parameters())[-2][1].shape[-1], 1)
        
    def forward(self, feature):
        x = self.model(feature)
        return self.regressor(x)

In [35]:
def train_fn(model, dl, criterion, optimizer, scheduler, grad_scaler):
    model.train()
    metrics = {
        "loss": AverageMeter("loss", fmt=":.4f"),
    }
    
    for idx, batch in enumerate(dl):
        output_topic = model(batch["feature"])
        loss = criterion(batch["label"], output_topic.flatten())
        # initialization gradients to zero
        optimizer.zero_grad()
        # backprop
        loss.backward()
        # update optimizer
        optimizer.step()
        metrics["loss"].update(loss.item())
        
        if CFG.debug:
            if idx >= 4: break
    
    # update learning rate scheduler
    scheduler.step()    

    return metrics

def valid_fn(model, dl, criterion):
    model.eval()
    metrics = {
        "loss": AverageMeter("loss", fmt=":.6f"),
    }   
    for idx, batch in enumerate(dl):
        with torch.no_grad():
            output_topic = model(batch["feature"])
            loss = criterion(batch["label"], output_topic.flatten())
        # update metric value
        metrics["loss"].update(loss.item())
    
        if CFG.debug:
            if idx >= 4: break
            
    return metrics

def do_fold_training(BEST_PARAMS={}):
        
    if len(BEST_PARAMS) == 0:
        BEST_PARAMS["nfeatures"] = full_x.shape[-1]
        BEST_PARAMS["dropoutRate"] = 0.5
        BEST_PARAMS["hidden_layers"] = 32
        BEST_PARAMS["layer_depth"] = 2
        BEST_PARAMS["nblocks"] = 2
    else:
        BEST_PARAMS["nfeatures"] = full_x.shape[-1]
        
    model = CustomModel(BEST_PARAMS)
    train_dl = DataLoader(CustomDataset(torch.from_numpy(train_x).to(torch.float32), torch.from_numpy(train_y).to(torch.float32)), shuffle=True, batch_size=CFG.batch_size, drop_last=True)
    valid_dl = DataLoader(CustomDataset(torch.from_numpy(valid_x).to(torch.float32), torch.from_numpy(valid_y).to(torch.float32)), shuffle=False, batch_size=CFG.batch_size)

    # set optimizer
    optimizer_parameters = get_optimizer_params(
        model,
        eta=CFG.eta,
        weight_decay=CFG.weight_decay
    )
    optimizer = AdamW(optimizer_parameters, lr=CFG.eta, weight_decay=CFG.weight_decay)
    scheduler = get_scheduler(
        optimizer,
        num_warmup_steps=0,
        num_training_steps=CFG.epochs
    )
#     scheduler = None
    grad_scaler = torch.cuda.amp.GradScaler() if torch.cuda.is_available() else None
    criterion = nn.MSELoss(reduction="mean")
    best_score = np.inf

    early_stopping_cnt = 0
    for epoch in range(CFG.epochs):
        epoch_start_time = time.time()
        # training
        train_metrics = train_fn(model, train_dl, criterion, optimizer, scheduler, grad_scaler)
        # evaluation
        valid_metrics = valid_fn(model, valid_dl, criterion)

        score = valid_metrics["loss"].avg
    
        if score < best_score:
            best_score = score
            return_score_dic = {
                "fold": fold,
                "train_loss": train_metrics["loss"].avg,
                "valid_loss": valid_metrics["loss"].avg,
            }
            model_save_dic = {'model': model.state_dict()}
            early_stopping_cnt = 0
        else:
            early_stopping_cnt += 1
        
        if early_stopping_cnt == CFG.early_stopping_rounds:
            break
        
    torch.save(
        model_save_dic,
        f"./model_fold{fold}_best.pth",
    )
    model.load_state_dict(torch.load(f"./model_fold{fold}_best.pth")["model"])
    model.eval()

    return copy.deepcopy(model), return_score_dic

In [36]:
# fake_params = {}
# fake_params["nfeatures"] = full_x.shape[-1]
# fake_params["dropoutRate"] = 0.5
# fake_params["hidden_layers"] = 32
# fake_params["layer_depth"] = 2
# fake_params["nblocks"] = 2

# CustomModel(fake_params)

In [37]:
line_split = [
    # A
    ["A_31"],
    # O, T
    ["T_31", "O_31"],
]
score_dic = {
    "mae": None,
    "r2": None,
    "accuracy": None,
    "f1": None,
}
line_split_f1 = {}
line_score = {"_".join(i): {"train": score_dic.copy(), "valid": score_dic.copy()} for i in line_split}
valid_pred = {"_".join(i): None for i in line_split}
test_pred = {"_".join(i): None for i in line_split}

for line in line_split:
    seed_everything()
    
    # Training
    # === Preprocessing (Train) ===
    full_x = df_full[df_full["product_code"].isin(line)]
    full_x["line"] = lbe.transform(full_x["line"])
#     full_x = full_x.drop_duplicates().reset_index(drop=True)
    
    # Set target
    full_y = full_x["y_quality"].values
    full_y_cls = full_x["y_class"].values
    # Drop columns
    full_x = full_x.drop(["product_id", "y_class", "y_quality", "timestamp", "product_code"], axis=1)
    
    full_x = full_x[full_x.columns[~(full_x.var() == 0).values & ~full_x.isna().all().values]]
    full_x = full_x.fillna(-1.0)
    full_x = full_x.T.drop_duplicates().T
    
    selected_vars = full_x.columns
    cat_vars = ["line"]
    num_vars = diff(selected_vars, cat_vars)
    categoIdx = findIdx(selected_vars, cat_vars)

    full_x[num_vars] = full_x[num_vars].astype("float32")
    full_x[cat_vars] = full_x[cat_vars].astype("int32")
    
    scaler = MinMaxScaler()
    full_x = scaler.fit_transform(full_x)
    print(full_x.shape)
    
    # === Preprocessing (Test) ===
    test_x = df_test[df_test["product_code"].isin(line)]
    test_x["line"] = lbe.transform(test_x["line"])
    test_x = test_x[selected_vars]
    test_x = test_x.fillna(-1.0)
    
    test_x[num_vars] = test_x[num_vars].astype("float32")
    test_x[cat_vars] = test_x[cat_vars].astype("int32")
    test_x = scaler.transform(test_x)
#     break
    
    # === Neural Architecture Search with Optuna & Training ===
    line_full_pred = np.zeros(len(full_x))
    line_test_pred = np.zeros(len(test_x))
    kfolds_spliter = StratifiedKFold(3 if CFG.debug else 5, shuffle=True, random_state=42)
    for fold, (train_idx, valid_idx) in enumerate(kfolds_spliter.split(full_x, full_y_cls)):

        train_x = full_x[train_idx]
        train_y = np.log1p(full_y[train_idx])
        valid_x = full_x[valid_idx]
        valid_y = np.log1p(full_y[valid_idx])

        output_container = {"model": None, "optuna_score": np.inf}
        optuna_direction = "minimize"
        optuna_trials = 3 if CFG.debug else 100
        optuna_earlyStopping = Optuna_EarlyStoppingCallback(max(5, 100), direction=optuna_direction)
        optuna_timout = 6 * 3600
        optuna_study = create_study(direction=optuna_direction, sampler=TPESampler(seed=GLOBAL_SEED))

        optuna_study.optimize(
            lambda trial: optuna_objective_function(
                trial, GLOBAL_SEED,
                train_x, train_y, valid_x, valid_y, output_container=output_container,
            ),
            n_jobs=1, n_trials=optuna_trials, timeout=optuna_timout, callbacks=[optuna_earlyStopping]
        )

        model = output_container["model"]
        
        tmp_pred = []
        dl = DataLoader(CustomDataset(torch.from_numpy(valid_x).to(torch.float32), torch.zeros(len(valid_x)).to(torch.float32)), shuffle=False, batch_size=CFG.batch_size)
        for batch in dl:
            with torch.no_grad():
                tmp_pred.append(model(batch["feature"]).detach().cpu().numpy().flatten())
        tmp_pred = np.concatenate(tmp_pred, axis=0, dtype="float32")
        line_full_pred[valid_idx] = np.expm1(tmp_pred)
        
        tmp_pred = []
        dl = DataLoader(CustomDataset(torch.from_numpy(test_x).to(torch.float32), torch.zeros(len(test_x)).to(torch.float32)), shuffle=False, batch_size=CFG.batch_size)
        for batch in dl:
            with torch.no_grad():
                tmp_pred.append(model(batch["feature"]).detach().cpu().numpy().flatten())
        tmp_pred = np.concatenate(tmp_pred, axis=0, dtype="float32")
        line_test_pred[:] += np.expm1(tmp_pred) / kfolds_spliter.n_splits

    # Evaluation
    y_pred = line_full_pred.copy()
    y_true = full_y.copy()
    eval_mae = mean_absolute_error(y_true, y_pred)
    eval_r2 = r2_score(y_true, y_pred)

    # Transform quality to class
    y_true_class = full_y_cls.copy()
    y_pred_class = quality_to_class(y_pred, {0: 0.52507, 1: 0.53490})
    eval_acc = accuracy_score(y_true_class, y_pred_class)
    eval_f1 = f1_score(y_true_class, y_pred_class, average="macro")
    
    # Save values
    line_score["_".join(line)]["valid"]["mae"] = eval_mae
    line_score["_".join(line)]["valid"]["r2"] = eval_r2
    line_score["_".join(line)]["valid"]["accuracy"] = eval_acc
    line_score["_".join(line)]["valid"]["f1"] = eval_f1
    valid_pred["_".join(line)] = y_pred
    
    # Inference
    y_pred = line_test_pred.copy()
    
    # Save values
    test_pred["_".join(line)] = y_pred

df_meta["reg_qual"] = [valid_pred, test_pred]

(249, 1173)


[I 2023-02-17 13:06:39,687] A new study created in memory with name: no-name-6f5fd81e-2d5f-4200-902c-16c703f195ca
[I 2023-02-17 13:06:52,632] Trial 0 finished with value: 0.00012133407430015862 and parameters: {'dropoutRate': 0.42472407130841755, 'hidden_layers': 10, 'layer_depth': 2, 'nblocks': 7, 'batch_norm': True}. Best is trial 0 with value: 0.00012133407430015862.
[I 2023-02-17 13:07:00,716] Trial 1 finished with value: 5.186214429368892e-05 and parameters: {'dropoutRate': 0.2935967122017216, 'hidden_layers': 2, 'layer_depth': 2, 'nblocks': 8, 'batch_norm': False}. Best is trial 1 with value: 5.186214429368892e-05.
[I 2023-02-17 13:07:16,996] Trial 2 finished with value: 0.0006123599059020097 and parameters: {'dropoutRate': 0.21235069657748148, 'hidden_layers': 10, 'layer_depth': 2, 'nblocks': 7, 'batch_norm': True}. Best is trial 1 with value: 5.186214429368892e-05.
[I 2023-02-17 13:07:30,864] Trial 3 finished with value: 0.02164161069175372 and parameters: {'dropoutRate': 0.310

[I 2023-02-17 13:10:18,483] Trial 30 finished with value: 5.045042822670299e-05 and parameters: {'dropoutRate': 0.3680698093950225, 'hidden_layers': 6, 'layer_depth': 2, 'nblocks': 6, 'batch_norm': False}. Best is trial 30 with value: 5.045042822670299e-05.
[I 2023-02-17 13:10:24,884] Trial 31 finished with value: 5.186139466017127e-05 and parameters: {'dropoutRate': 0.3806182959951603, 'hidden_layers': 6, 'layer_depth': 2, 'nblocks': 6, 'batch_norm': False}. Best is trial 30 with value: 5.045042822670299e-05.
[I 2023-02-17 13:10:29,937] Trial 32 finished with value: 5.1788656457154924e-05 and parameters: {'dropoutRate': 0.3943945930957188, 'hidden_layers': 6, 'layer_depth': 2, 'nblocks': 6, 'batch_norm': False}. Best is trial 30 with value: 5.045042822670299e-05.
[I 2023-02-17 13:10:35,077] Trial 33 finished with value: 5.157118159961381e-05 and parameters: {'dropoutRate': 0.40172011513344363, 'hidden_layers': 6, 'layer_depth': 2, 'nblocks': 6, 'batch_norm': False}. Best is trial 30 w

[I 2023-02-17 13:13:39,985] Trial 61 finished with value: 4.7085263584169574e-05 and parameters: {'dropoutRate': 0.4844894211228035, 'hidden_layers': 5, 'layer_depth': 2, 'nblocks': 6, 'batch_norm': False}. Best is trial 42 with value: 4.411377365333745e-05.
[I 2023-02-17 13:13:45,396] Trial 62 finished with value: 4.400542100204173e-05 and parameters: {'dropoutRate': 0.5322111060471312, 'hidden_layers': 5, 'layer_depth': 2, 'nblocks': 5, 'batch_norm': False}. Best is trial 62 with value: 4.400542100204173e-05.
[I 2023-02-17 13:13:50,798] Trial 63 finished with value: 0.0002754069188925384 and parameters: {'dropoutRate': 0.5395084170664127, 'hidden_layers': 3, 'layer_depth': 2, 'nblocks': 5, 'batch_norm': False}. Best is trial 62 with value: 4.400542100204173e-05.
[I 2023-02-17 13:13:56,109] Trial 64 finished with value: 4.369994240523038e-05 and parameters: {'dropoutRate': 0.5073682206008092, 'hidden_layers': 5, 'layer_depth': 2, 'nblocks': 5, 'batch_norm': False}. Best is trial 64 wi

[I 2023-02-17 13:16:31,743] Trial 92 finished with value: 4.394005444597636e-05 and parameters: {'dropoutRate': 0.4677665745320811, 'hidden_layers': 5, 'layer_depth': 2, 'nblocks': 5, 'batch_norm': False}. Best is trial 76 with value: 4.3699813956594044e-05.
[I 2023-02-17 13:16:37,164] Trial 93 finished with value: 4.2920114642933746e-05 and parameters: {'dropoutRate': 0.45777276341636103, 'hidden_layers': 5, 'layer_depth': 2, 'nblocks': 5, 'batch_norm': False}. Best is trial 93 with value: 4.2920114642933746e-05.
[I 2023-02-17 13:16:42,476] Trial 94 finished with value: 0.00024835981621261174 and parameters: {'dropoutRate': 0.4652199250624144, 'hidden_layers': 4, 'layer_depth': 2, 'nblocks': 5, 'batch_norm': False}. Best is trial 93 with value: 4.2920114642933746e-05.
[I 2023-02-17 13:16:47,904] Trial 95 finished with value: 6.748991052938017e-05 and parameters: {'dropoutRate': 0.4538382970381107, 'hidden_layers': 6, 'layer_depth': 2, 'nblocks': 5, 'batch_norm': False}. Best is trial 

...

[I 2023-02-17 14:55:45,963] Trial 69 finished with value: 1.432476564231264e-05 and parameters: {'dropoutRate': 0.4372175319690077, 'hidden_layers': 5, 'layer_depth': 2, 'nblocks': 7, 'batch_norm': False}. Best is trial 21 with value: 8.579216149150347e-06.
[I 2023-02-17 14:55:52,138] Trial 70 finished with value: 9.078794856047049e-06 and parameters: {'dropoutRate': 0.4012656811967991, 'hidden_layers': 6, 'layer_depth': 2, 'nblocks': 7, 'batch_norm': False}. Best is trial 21 with value: 8.579216149150347e-06.
[I 2023-02-17 14:55:58,355] Trial 71 finished with value: 9.07635603968831e-06 and parameters: {'dropoutRate': 0.400016380485717, 'hidden_layers': 6, 'layer_depth': 2, 'nblocks': 7, 'batch_norm': False}. Best is trial 21 with value: 8.579216149150347e-06.
[I 2023-02-17 14:56:04,546] Trial 72 finished with value: 9.09751635605163e-06 and parameters: {'dropoutRate': 0.396158588031939, 'hidden_layers': 6, 'layer_depth': 2, 'nblocks': 7, 'batch_norm': False}. Best is trial 21 with va

[I 2023-02-17 14:58:42,577] A new study created in memory with name: no-name-0b5ba0ce-d7ca-4cda-9ebd-ea11ebcc2e4b
[I 2023-02-17 14:58:58,203] Trial 0 finished with value: 0.06992909953826004 and parameters: {'dropoutRate': 0.42472407130841755, 'hidden_layers': 10, 'layer_depth': 2, 'nblocks': 7, 'batch_norm': True}. Best is trial 0 with value: 0.06992909953826004.
[I 2023-02-17 14:59:05,250] Trial 1 finished with value: 9.960983536277329e-05 and parameters: {'dropoutRate': 0.2935967122017216, 'hidden_layers': 2, 'layer_depth': 2, 'nblocks': 8, 'batch_norm': False}. Best is trial 1 with value: 9.960983536277329e-05.
[I 2023-02-17 14:59:20,942] Trial 2 finished with value: 0.10566909005865455 and parameters: {'dropoutRate': 0.21235069657748148, 'hidden_layers': 10, 'layer_depth': 2, 'nblocks': 7, 'batch_norm': True}. Best is trial 1 with value: 9.960983536277329e-05.
[I 2023-02-17 14:59:32,764] Trial 3 finished with value: 0.0024392991846171855 and parameters: {'dropoutRate': 0.310042705

[I 2023-02-17 15:01:54,022] Trial 30 finished with value: 1.1375637620226877e-05 and parameters: {'dropoutRate': 0.6397123139920761, 'hidden_layers': 5, 'layer_depth': 2, 'nblocks': 7, 'batch_norm': False}. Best is trial 24 with value: 1.059587383641277e-05.
[I 2023-02-17 15:02:00,299] Trial 31 finished with value: 1.489515853513189e-05 and parameters: {'dropoutRate': 0.7969102359115765, 'hidden_layers': 6, 'layer_depth': 2, 'nblocks': 7, 'batch_norm': False}. Best is trial 24 with value: 1.059587383641277e-05.
[I 2023-02-17 15:02:07,408] Trial 32 finished with value: 1.059302704852093e-05 and parameters: {'dropoutRate': 0.7481354571901968, 'hidden_layers': 7, 'layer_depth': 2, 'nblocks': 8, 'batch_norm': False}. Best is trial 32 with value: 1.059302704852093e-05.
[I 2023-02-17 15:02:14,525] Trial 33 finished with value: 1.0593324759611278e-05 and parameters: {'dropoutRate': 0.7164608894168316, 'hidden_layers': 7, 'layer_depth': 2, 'nblocks': 8, 'batch_norm': False}. Best is trial 32 w

[I 2023-02-17 15:07:30,439] Trial 61 finished with value: 1.0595935533628006e-05 and parameters: {'dropoutRate': 0.6711126963554934, 'hidden_layers': 7, 'layer_depth': 2, 'nblocks': 8, 'batch_norm': False}. Best is trial 38 with value: 1.0562043608312402e-05.
[I 2023-02-17 15:07:37,522] Trial 62 finished with value: 1.0593505455365934e-05 and parameters: {'dropoutRate': 0.6871757408904579, 'hidden_layers': 7, 'layer_depth': 2, 'nblocks': 8, 'batch_norm': False}. Best is trial 38 with value: 1.0562043608312402e-05.
[I 2023-02-17 15:07:44,571] Trial 63 finished with value: 1.0593470995622232e-05 and parameters: {'dropoutRate': 0.7206597350315351, 'hidden_layers': 7, 'layer_depth': 2, 'nblocks': 8, 'batch_norm': False}. Best is trial 38 with value: 1.0562043608312402e-05.
[I 2023-02-17 15:07:49,862] Trial 64 finished with value: 2.087168410172227e-05 and parameters: {'dropoutRate': 0.753413199763662, 'hidden_layers': 6, 'layer_depth': 2, 'nblocks': 8, 'batch_norm': False}. Best is trial 3

[I 2023-02-17 15:12:26,318] Trial 92 finished with value: 1.0596188114201746e-05 and parameters: {'dropoutRate': 0.6370177350677214, 'hidden_layers': 7, 'layer_depth': 2, 'nblocks': 8, 'batch_norm': True}. Best is trial 88 with value: 1.0537129605204933e-05.
[I 2023-02-17 15:12:44,624] Trial 93 finished with value: 1.059703934124122e-05 and parameters: {'dropoutRate': 0.5594227002200824, 'hidden_layers': 7, 'layer_depth': 2, 'nblocks': 8, 'batch_norm': True}. Best is trial 88 with value: 1.0537129605204933e-05.
[I 2023-02-17 15:13:02,784] Trial 94 finished with value: 0.00043910158759293455 and parameters: {'dropoutRate': 0.374581370745972, 'hidden_layers': 6, 'layer_depth': 2, 'nblocks': 8, 'batch_norm': True}. Best is trial 88 with value: 1.0537129605204933e-05.
[I 2023-02-17 15:13:21,013] Trial 95 finished with value: 0.26942756440904403 and parameters: {'dropoutRate': 0.7015418227685948, 'hidden_layers': 8, 'layer_depth': 2, 'nblocks': 8, 'batch_norm': True}. Best is trial 88 with 

In [38]:
print(line_score)

{'A_31': {'train': {'mae': None, 'r2': None, 'accuracy': None, 'f1': None}, 'valid': {'mae': 0.006915394920946724, 'r2': 0.10123637151343923, 'accuracy': 0.5220883534136547, 'f1': 0.45546831828420276}}, 'T_31_O_31': {'train': {'mae': None, 'r2': None, 'accuracy': None, 'f1': None}, 'valid': {'mae': 0.0033241467109032213, 'r2': -0.0877467458400698, 'accuracy': 0.828080229226361, 'f1': 0.3241090146750524}}}


### Training - Regression (y_class)

In [39]:
class CustomModel(nn.Module):
    def __init__(self, params):
        super(CustomModel, self).__init__()
        self.params = params
        
        self.model = nn.Sequential(
            *[ResidualLinearBlock(
                params["nfeatures"] if i == 0 else max(4, int(params["hidden_layers"] / (2**(i-1)))),
                dropoutRate=params["dropoutRate"],
                hidden_layers=max(4, int(params["hidden_layers"] / (2**i))),
                layer_depth=params["layer_depth"],
                batch_norm=params["batch_norm"]) for i in range(params["nblocks"])
             ],
        )

        self.regressor = nn.Linear(list(self.model.named_parameters())[-2][1].shape[-1], 1)
        
    def forward(self, feature):
        x = self.model(feature)
        return self.regressor(x)

In [40]:
def train_fn(model, dl, criterion, optimizer, scheduler, grad_scaler):
    model.train()
    metrics = {
        "loss": AverageMeter("loss", fmt=":.4f"),
    }
    
    for idx, batch in enumerate(dl):
        output_topic = model(batch["feature"])
        loss = criterion(batch["label"], output_topic.flatten())
        # initialization gradients to zero
        optimizer.zero_grad()
        # backprop
        loss.backward()
        # update optimizer
        optimizer.step()
        metrics["loss"].update(loss.item())
        
        if CFG.debug:
            if idx >= 4: break
                
    # update learning rate scheduler
    scheduler.step()

    return metrics

def valid_fn(model, dl, criterion):
    model.eval()
    metrics = {
        "loss": AverageMeter("loss", fmt=":.6f"),
    }   
    for idx, batch in enumerate(dl):
        with torch.no_grad():
            output_topic = model(batch["feature"])
            loss = criterion(batch["label"], output_topic.flatten())
        # update metric value
        metrics["loss"].update(loss.item())
    
        if CFG.debug:
            if idx >= 4: break
            
    return metrics

def do_fold_training(BEST_PARAMS={}):
        
    if len(BEST_PARAMS) == 0:
        BEST_PARAMS["nfeatures"] = full_x.shape[-1]
        BEST_PARAMS["dropoutRate"] = 0.5
        BEST_PARAMS["hidden_layers"] = 32
        BEST_PARAMS["layer_depth"] = 2
        BEST_PARAMS["nblocks"] = 2
    else:
        BEST_PARAMS["nfeatures"] = full_x.shape[-1]
        
    model = CustomModel(BEST_PARAMS)
    train_dl = DataLoader(CustomDataset(torch.from_numpy(train_x).to(torch.float32), torch.from_numpy(train_y).to(torch.float32)), shuffle=True, batch_size=CFG.batch_size, drop_last=True)
    valid_dl = DataLoader(CustomDataset(torch.from_numpy(valid_x).to(torch.float32), torch.from_numpy(valid_y).to(torch.float32)), shuffle=False, batch_size=CFG.batch_size)

    # set optimizer
    optimizer_parameters = get_optimizer_params(
        model,
        eta=CFG.eta,
        weight_decay=CFG.weight_decay
    )
    optimizer = AdamW(optimizer_parameters, lr=CFG.eta, weight_decay=CFG.weight_decay)
    scheduler = get_scheduler(
        optimizer,
        num_warmup_steps=0,
        num_training_steps=CFG.epochs
    )
#     scheduler = None
    grad_scaler = torch.cuda.amp.GradScaler() if torch.cuda.is_available() else None
    criterion = nn.MSELoss(reduction="mean")
    best_score = np.inf

    early_stopping_cnt = 0
    for epoch in range(CFG.epochs):
        epoch_start_time = time.time()
        # training
        train_metrics = train_fn(model, train_dl, criterion, optimizer, scheduler, grad_scaler)
        # evaluation
        valid_metrics = valid_fn(model, valid_dl, criterion)

        score = valid_metrics["loss"].avg
    
        if score < best_score:
            best_score = score
            return_score_dic = {
                "fold": fold,
                "train_loss": train_metrics["loss"].avg,
                "valid_loss": valid_metrics["loss"].avg,
            }
            model_save_dic = {'model': model.state_dict()}
            early_stopping_cnt = 0
        else:
            early_stopping_cnt += 1
        
        if early_stopping_cnt == CFG.early_stopping_rounds:
            break
        
    torch.save(
        model_save_dic,
        f"./model_fold{fold}_best.pth",
    )
    model.load_state_dict(torch.load(f"./model_fold{fold}_best.pth")["model"])
    model.eval()

    return copy.deepcopy(model), return_score_dic

In [41]:
# fake_params = {}
# fake_params["nfeatures"] = full_x.shape[-1]
# fake_params["dropoutRate"] = 0.5
# fake_params["hidden_layers"] = 32
# fake_params["layer_depth"] = 2
# fake_params["nblocks"] = 4
# fake_params["batch_norm"] = True

# CustomModel(fake_params)

In [42]:
line_split = [
    # A
    ["A_31"],
    # O, T
    ["T_31", "O_31"],
]
score_dic = {
    "mae": None,
    "r2": None,
    "accuracy": None,
    "f1": None,
}
line_split_f1 = {}
line_score = {"_".join(i): {"train": score_dic.copy(), "valid": score_dic.copy()} for i in line_split}
valid_pred = {"_".join(i): None for i in line_split}
test_pred = {"_".join(i): None for i in line_split}

for line in line_split:
    seed_everything()
    # Training
    # === Preprocessing (Train) ===
    full_x = df_full[df_full["product_code"].isin(line)]
    full_x["line"] = lbe.transform(full_x["line"])
#     full_x = full_x.drop_duplicates().reset_index(drop=True)
    
    # Set target
    full_y = full_x["y_class"].values
    full_y_cls = full_x["y_class"].values
    # Drop columns
    full_x = full_x.drop(["product_id", "y_class", "y_quality", "timestamp", "product_code"], axis=1)
    
    full_x = full_x[full_x.columns[~(full_x.var() == 0).values & ~full_x.isna().all().values]]
    full_x = full_x.fillna(-1.0)
    full_x = full_x.T.drop_duplicates().T
    
    selected_vars = full_x.columns
    cat_vars = ["line"]
    num_vars = diff(selected_vars, cat_vars)
    categoIdx = findIdx(selected_vars, cat_vars)

    full_x[num_vars] = full_x[num_vars].astype("float32")
    full_x[cat_vars] = full_x[cat_vars].astype("int32")
    
    scaler = MinMaxScaler()
    full_x = scaler.fit_transform(full_x)
    print(full_x.shape)
    
    # === Preprocessing (Test) ===
    test_x = df_test[df_test["product_code"].isin(line)]
    test_x["line"] = lbe.transform(test_x["line"])
    test_x = test_x[selected_vars]
    test_x = test_x.fillna(-1.0)
    
    test_x[num_vars] = test_x[num_vars].astype("float32")
    test_x[cat_vars] = test_x[cat_vars].astype("int32")
    test_x = scaler.transform(test_x)
#     break

    # === Neural Architecture Search with Optuna & Training ===
    line_full_pred = np.zeros(len(full_x))
    line_test_pred = np.zeros(len(test_x))
    kfolds_spliter = StratifiedKFold(3 if CFG.debug else 5, shuffle=True, random_state=42)
    for fold, (train_idx, valid_idx) in enumerate(kfolds_spliter.split(full_x, full_y_cls)):

        train_x = full_x[train_idx]
        train_y = full_y[train_idx]
        valid_x = full_x[valid_idx]
        valid_y = full_y[valid_idx]

        output_container = {"model": None, "optuna_score": np.inf}
        optuna_direction = "minimize"
        optuna_trials = 3 if CFG.debug else 100
        optuna_earlyStopping = Optuna_EarlyStoppingCallback(max(5, 100), direction=optuna_direction)
        optuna_timout = 6 * 3600
        optuna_study = create_study(direction=optuna_direction, sampler=TPESampler(seed=GLOBAL_SEED))

        optuna_study.optimize(
            lambda trial: optuna_objective_function(
                trial, GLOBAL_SEED,
                train_x, train_y, valid_x, valid_y, output_container=output_container,
            ),
            n_jobs=1, n_trials=optuna_trials, timeout=optuna_timout, callbacks=[optuna_earlyStopping]
        )

        model = output_container["model"]
        
        tmp_pred = []
        dl = DataLoader(CustomDataset(torch.from_numpy(valid_x).to(torch.float32), torch.zeros(len(valid_x)).to(torch.float32)), shuffle=False, batch_size=CFG.batch_size)
        for batch in dl:
            with torch.no_grad():
                tmp_pred.append(model(batch["feature"]).detach().cpu().numpy().flatten())
        tmp_pred = np.concatenate(tmp_pred, axis=0, dtype="float32")
        line_full_pred[valid_idx] = tmp_pred
        
        tmp_pred = []
        dl = DataLoader(CustomDataset(torch.from_numpy(test_x).to(torch.float32), torch.zeros(len(test_x)).to(torch.float32)), shuffle=False, batch_size=CFG.batch_size)
        for batch in dl:
            with torch.no_grad():
                tmp_pred.append(model(batch["feature"]).detach().cpu().numpy().flatten())
        tmp_pred = np.concatenate(tmp_pred, axis=0, dtype="float32")
        line_test_pred[:] += tmp_pred / kfolds_spliter.n_splits
    
    # Evaluation
    y_true = full_y.copy()
    y_pred = line_full_pred.copy()
    eval_mae = mean_absolute_error(y_true, y_pred)
    eval_r2 = r2_score(y_true, y_pred)

    # Transform quality to class
    y_true_class = full_y_cls.copy()
    y_pred_class = np.clip(np.round(y_true), 0, 2)
    eval_acc = accuracy_score(y_true_class, full_y_cls)
    eval_f1 = f1_score(y_true_class, full_y_cls, average="macro")
    
    # Save values
    line_score["_".join(line)]["valid"]["mae"] = eval_mae
    line_score["_".join(line)]["valid"]["r2"] = eval_r2
    line_score["_".join(line)]["valid"]["accuracy"] = eval_acc
    line_score["_".join(line)]["valid"]["f1"] = eval_f1
    valid_pred["_".join(line)] = y_pred
    
    # Inference
    y_pred = line_test_pred.copy()
    
    # Save values
    test_pred["_".join(line)] = y_pred

df_meta["reg_cls"] = [valid_pred, test_pred]

[I 2023-02-17 15:14:18,548] A new study created in memory with name: no-name-54141cfd-fa1d-4922-8387-eb8244d5e7f7


(249, 1173)


[I 2023-02-17 15:14:29,858] Trial 0 finished with value: 0.5219478194530194 and parameters: {'dropoutRate': 0.42472407130841755, 'hidden_layers': 10, 'layer_depth': 2, 'nblocks': 7, 'batch_norm': True}. Best is trial 0 with value: 0.5219478194530194.
[I 2023-02-17 15:14:34,888] Trial 1 finished with value: 0.5164155914233282 and parameters: {'dropoutRate': 0.2935967122017216, 'hidden_layers': 2, 'layer_depth': 2, 'nblocks': 8, 'batch_norm': False}. Best is trial 1 with value: 0.5164155914233282.
[I 2023-02-17 15:14:45,937] Trial 2 finished with value: 0.5648777026396531 and parameters: {'dropoutRate': 0.21235069657748148, 'hidden_layers': 10, 'layer_depth': 2, 'nblocks': 7, 'batch_norm': True}. Best is trial 1 with value: 0.5164155914233282.
[I 2023-02-17 15:14:54,407] Trial 3 finished with value: 0.6415330102810493 and parameters: {'dropoutRate': 0.3100427059120603, 'hidden_layers': 4, 'layer_depth': 2, 'nblocks': 5, 'batch_norm': True}. Best is trial 1 with value: 0.5164155914233282.

[I 2023-02-17 15:16:13,701] Trial 31 finished with value: 0.40177392959594727 and parameters: {'dropoutRate': 0.4591146715998062, 'hidden_layers': 6, 'layer_depth': 2, 'nblocks': 2, 'batch_norm': False}. Best is trial 27 with value: 0.40138663122287166.
[I 2023-02-17 15:16:15,407] Trial 32 finished with value: 0.4046918380145843 and parameters: {'dropoutRate': 0.4382825641819016, 'hidden_layers': 6, 'layer_depth': 2, 'nblocks': 2, 'batch_norm': False}. Best is trial 27 with value: 0.40138663122287166.
[I 2023-02-17 15:16:17,097] Trial 33 finished with value: 0.4277249649167061 and parameters: {'dropoutRate': 0.4442169563661256, 'hidden_layers': 7, 'layer_depth': 2, 'nblocks': 2, 'batch_norm': False}. Best is trial 27 with value: 0.40138663122287166.
[I 2023-02-17 15:16:22,142] Trial 34 finished with value: 0.5133362332215676 and parameters: {'dropoutRate': 0.34520041681662006, 'hidden_layers': 6, 'layer_depth': 2, 'nblocks': 8, 'batch_norm': False}. Best is trial 27 with value: 0.40138

[I 2023-02-17 15:17:31,863] Trial 62 finished with value: 0.4001663694014916 and parameters: {'dropoutRate': 0.4367172396855386, 'hidden_layers': 6, 'layer_depth': 2, 'nblocks': 2, 'batch_norm': False}. Best is trial 61 with value: 0.3959673849435953.
[I 2023-02-17 15:17:33,575] Trial 63 finished with value: 0.39496847643302035 and parameters: {'dropoutRate': 0.401080912918325, 'hidden_layers': 6, 'layer_depth': 2, 'nblocks': 2, 'batch_norm': False}. Best is trial 63 with value: 0.39496847643302035.
[I 2023-02-17 15:17:36,909] Trial 64 finished with value: 0.45258523523807526 and parameters: {'dropoutRate': 0.37744384206016607, 'hidden_layers': 5, 'layer_depth': 2, 'nblocks': 5, 'batch_norm': False}. Best is trial 63 with value: 0.39496847643302035.
[I 2023-02-17 15:17:38,626] Trial 65 finished with value: 0.4002300300277196 and parameters: {'dropoutRate': 0.4047466021474217, 'hidden_layers': 6, 'layer_depth': 2, 'nblocks': 2, 'batch_norm': False}. Best is trial 63 with value: 0.394968

...

[I 2023-02-17 16:01:36,326] Trial 72 finished with value: 0.1467205492598522 and parameters: {'dropoutRate': 0.36788981969139756, 'hidden_layers': 2, 'layer_depth': 2, 'nblocks': 2, 'batch_norm': False}. Best is trial 56 with value: 0.1457575613555188.
[I 2023-02-17 16:01:38,678] Trial 73 finished with value: 0.14662274930419195 and parameters: {'dropoutRate': 0.5865769431459438, 'hidden_layers': 2, 'layer_depth': 2, 'nblocks': 2, 'batch_norm': False}. Best is trial 56 with value: 0.1457575613555188.
[I 2023-02-17 16:01:40,991] Trial 74 finished with value: 0.14639556772211412 and parameters: {'dropoutRate': 0.5654079034554775, 'hidden_layers': 3, 'layer_depth': 2, 'nblocks': 2, 'batch_norm': False}. Best is trial 56 with value: 0.1457575613555188.
[I 2023-02-17 16:01:43,297] Trial 75 finished with value: 0.14581593447494218 and parameters: {'dropoutRate': 0.6078189682415807, 'hidden_layers': 4, 'layer_depth': 2, 'nblocks': 2, 'batch_norm': False}. Best is trial 56 with value: 0.145757

[I 2023-02-17 16:03:44,282] Trial 3 finished with value: 0.26201279228553176 and parameters: {'dropoutRate': 0.3100427059120603, 'hidden_layers': 4, 'layer_depth': 2, 'nblocks': 5, 'batch_norm': True}. Best is trial 1 with value: 0.20482550089743906.
[I 2023-02-17 16:03:48,242] Trial 4 finished with value: 0.20512432119600513 and parameters: {'dropoutRate': 0.5671117368334277, 'hidden_layers': 3, 'layer_depth': 2, 'nblocks': 4, 'batch_norm': False}. Best is trial 1 with value: 0.20482550089743906.
[I 2023-02-17 16:03:59,876] Trial 5 finished with value: 0.2529126552657949 and parameters: {'dropoutRate': 0.6711055768358083, 'hidden_layers': 3, 'layer_depth': 2, 'nblocks': 5, 'batch_norm': True}. Best is trial 1 with value: 0.20482550089743906.
[I 2023-02-17 16:04:02,231] Trial 6 finished with value: 0.198603626819224 and parameters: {'dropoutRate': 0.5645269111408631, 'hidden_layers': 3, 'layer_depth': 2, 'nblocks': 2, 'batch_norm': False}. Best is trial 6 with value: 0.198603626819224.

[I 2023-02-17 16:05:47,726] Trial 34 finished with value: 0.20563533803619471 and parameters: {'dropoutRate': 0.623158797028926, 'hidden_layers': 4, 'layer_depth': 2, 'nblocks': 8, 'batch_norm': False}. Best is trial 15 with value: 0.19855954870581627.
[I 2023-02-17 16:05:52,849] Trial 35 finished with value: 0.33063393955429393 and parameters: {'dropoutRate': 0.5725101001467793, 'hidden_layers': 2, 'layer_depth': 2, 'nblocks': 2, 'batch_norm': True}. Best is trial 15 with value: 0.19855954870581627.
[I 2023-02-17 16:05:55,566] Trial 36 finished with value: 0.20565221211250495 and parameters: {'dropoutRate': 0.5545943115819566, 'hidden_layers': 5, 'layer_depth': 2, 'nblocks': 3, 'batch_norm': False}. Best is trial 15 with value: 0.19855954870581627.
[I 2023-02-17 16:06:08,812] Trial 37 finished with value: 0.2051124844260307 and parameters: {'dropoutRate': 0.4307125432696578, 'hidden_layers': 3, 'layer_depth': 2, 'nblocks': 6, 'batch_norm': True}. Best is trial 15 with value: 0.1985595

[I 2023-02-17 16:07:39,303] Trial 65 finished with value: 0.1958397772266633 and parameters: {'dropoutRate': 0.25239749872415207, 'hidden_layers': 6, 'layer_depth': 2, 'nblocks': 2, 'batch_norm': False}. Best is trial 44 with value: 0.19549864892097604.
[I 2023-02-17 16:07:41,671] Trial 66 finished with value: 0.1974497343439402 and parameters: {'dropoutRate': 0.2007875597242009, 'hidden_layers': 7, 'layer_depth': 2, 'nblocks': 2, 'batch_norm': False}. Best is trial 44 with value: 0.19549864892097604.
[I 2023-02-17 16:07:44,085] Trial 67 finished with value: 1.2668529782030318 and parameters: {'dropoutRate': 0.24321428889362282, 'hidden_layers': 5, 'layer_depth': 2, 'nblocks': 2, 'batch_norm': False}. Best is trial 44 with value: 0.19549864892097604.
[I 2023-02-17 16:07:47,240] Trial 68 finished with value: 0.5106351160340838 and parameters: {'dropoutRate': 0.3264421613308198, 'hidden_layers': 8, 'layer_depth': 2, 'nblocks': 3, 'batch_norm': False}. Best is trial 44 with value: 0.19549

[I 2023-02-17 16:09:05,152] Trial 96 finished with value: 0.1965957222920325 and parameters: {'dropoutRate': 0.26526819354750353, 'hidden_layers': 6, 'layer_depth': 2, 'nblocks': 2, 'batch_norm': False}. Best is trial 44 with value: 0.19549864892097604.
[I 2023-02-17 16:09:07,524] Trial 97 finished with value: 0.1962890255704729 and parameters: {'dropoutRate': 0.23087351347767118, 'hidden_layers': 6, 'layer_depth': 2, 'nblocks': 2, 'batch_norm': False}. Best is trial 44 with value: 0.19549864892097604.
[I 2023-02-17 16:09:09,965] Trial 98 finished with value: 1.2669464879565768 and parameters: {'dropoutRate': 0.21105184285745887, 'hidden_layers': 5, 'layer_depth': 2, 'nblocks': 2, 'batch_norm': False}. Best is trial 44 with value: 0.19549864892097604.
[I 2023-02-17 16:09:12,317] Trial 99 finished with value: 0.19739070096208403 and parameters: {'dropoutRate': 0.2324910730144752, 'hidden_layers': 7, 'layer_depth': 2, 'nblocks': 2, 'batch_norm': False}. Best is trial 44 with value: 0.195

In [43]:
print(line_score)

{'A_31': {'train': {'mae': None, 'r2': None, 'accuracy': None, 'f1': None}, 'valid': {'mae': 0.523805243064122, 'r2': 0.20171226716226553, 'accuracy': 1.0, 'f1': 1.0}}, 'T_31_O_31': {'train': {'mae': None, 'r2': None, 'accuracy': None, 'f1': None}, 'valid': {'mae': 0.20807391977241865, 'r2': 0.050596451110149765, 'accuracy': 1.0, 'f1': 1.0}}}


### Training - Classfication (y_class)

In [44]:
class CustomModel(nn.Module):
    def __init__(self, params):
        super(CustomModel, self).__init__()
        self.params = params
        
        self.model = nn.Sequential(
            *[ResidualLinearBlock(
                params["nfeatures"] if i == 0 else max(4, int(params["hidden_layers"] / (2**(i-1)))),
                dropoutRate=params["dropoutRate"],
                hidden_layers=max(4, int(params["hidden_layers"] / (2**i))),
                layer_depth=params["layer_depth"],
                batch_norm=params["batch_norm"]) for i in range(params["nblocks"])
             ],
        )

        self.regressor = nn.Linear(list(self.model.named_parameters())[-2][1].shape[-1], 3)
        
    def forward(self, feature):
        x = self.model(feature)
        return self.regressor(x)

In [45]:
def train_fn(model, dl, criterion, optimizer, scheduler, grad_scaler):
    model.train()
    metrics = {
        "loss": AverageMeter("loss", fmt=":.4f"),
    }
    for idx, batch in enumerate(dl):
        output_topic = model(batch["feature"])
        loss = criterion(F.softmax(output_topic, dim=-1), batch["label"].to(torch.int64))
        # initialization gradients to zero
        optimizer.zero_grad()
        # backprop
        loss.backward()
        # update optimizer
        optimizer.step()
        metrics["loss"].update(loss.item())
        if CFG.debug:
            if idx >= 4: break
                
    # update learning rate scheduler
    scheduler.step()
        
    return metrics

def valid_fn(model, dl, criterion):
    model.eval()
    metrics = {
        "loss": AverageMeter("loss", fmt=":.6f"),
    }   
    for idx, batch in enumerate(dl):
        with torch.no_grad():
            output_topic = model(batch["feature"])
            loss = criterion(F.softmax(output_topic, dim=-1), batch["label"].to(torch.int64))
        # update metric value
        metrics["loss"].update(loss.item())
    
        if CFG.debug:
            if idx >= 4: break
            
    return metrics

def do_fold_training(BEST_PARAMS={}):
        
    if len(BEST_PARAMS) == 0:
        BEST_PARAMS["nfeatures"] = full_x.shape[-1]
        BEST_PARAMS["dropoutRate"] = 0.5
        BEST_PARAMS["hidden_layers"] = 32
        BEST_PARAMS["layer_depth"] = 2
        BEST_PARAMS["nblocks"] = 2
    else:
        BEST_PARAMS["nfeatures"] = full_x.shape[-1]
        
    model = CustomModel(BEST_PARAMS)
    train_dl = DataLoader(CustomDataset(torch.from_numpy(train_x).to(torch.float32), torch.from_numpy(train_y).to(torch.float32)), shuffle=True, batch_size=CFG.batch_size, drop_last=True)
    valid_dl = DataLoader(CustomDataset(torch.from_numpy(valid_x).to(torch.float32), torch.from_numpy(valid_y).to(torch.float32)), shuffle=False, batch_size=CFG.batch_size)

    # set optimizer
    optimizer_parameters = get_optimizer_params(
        model,
        eta=CFG.eta,
        weight_decay=CFG.weight_decay
    )
    optimizer = AdamW(optimizer_parameters, lr=CFG.eta, weight_decay=CFG.weight_decay)
    scheduler = get_scheduler(
        optimizer,
        num_warmup_steps=0,
        num_training_steps=CFG.epochs
    )
#     scheduler = None
    grad_scaler = torch.cuda.amp.GradScaler() if torch.cuda.is_available() else None
    criterion = nn.CrossEntropyLoss(reduction="mean")
    best_score = np.inf

    early_stopping_cnt = 0
    for epoch in range(CFG.epochs):
        epoch_start_time = time.time()
        # training
        train_metrics = train_fn(model, train_dl, criterion, optimizer, scheduler, grad_scaler)
        # evaluation
        valid_metrics = valid_fn(model, valid_dl, criterion)

        score = valid_metrics["loss"].avg
    
        if score < best_score:
            best_score = score
            return_score_dic = {
                "fold": fold,
                "train_loss": train_metrics["loss"].avg,
                "valid_loss": valid_metrics["loss"].avg,
            }
            model_save_dic = {'model': model.state_dict()}
            early_stopping_cnt = 0
        else:
            early_stopping_cnt += 1
        
        if early_stopping_cnt == CFG.early_stopping_rounds:
            break
        
    torch.save(
        model_save_dic,
        f"./model_fold{fold}_best.pth",
    )
    model.load_state_dict(torch.load(f"./model_fold{fold}_best.pth")["model"])
    model.eval()

    return copy.deepcopy(model), return_score_dic

In [46]:
line_split = [
    # A
    ["A_31"],
    # O, T
    ["T_31", "O_31"],
]
score_dic = {
    "mae": None,
    "r2": None,
    "accuracy": None,
    "f1": None,
}
line_split_f1 = {}
line_score = {"_".join(i): {"train": score_dic.copy(), "valid": score_dic.copy()} for i in line_split}
valid_pred = {"_".join(i): None for i in line_split}
test_pred = {"_".join(i): None for i in line_split}

for line in line_split:
    seed_everything()
    # Training
    # === Preprocessing (Train) ===
    full_x = df_full[df_full["product_code"].isin(line)]
    full_x["line"] = lbe.transform(full_x["line"])
#     full_x = full_x.drop_duplicates().reset_index(drop=True)
    
    # Set target
    full_y = full_x["y_class"].values
    full_y_cls = full_x["y_class"].values
    # Drop columns
    full_x = full_x.drop(["product_id", "y_class", "y_quality", "timestamp", "product_code"], axis=1)
    
    full_x = full_x[full_x.columns[~(full_x.var() == 0).values & ~full_x.isna().all().values]]
    full_x = full_x.fillna(-1.0)
    full_x = full_x.T.drop_duplicates().T
    
    selected_vars = full_x.columns
    cat_vars = ["line"]
    num_vars = diff(selected_vars, cat_vars)
    categoIdx = findIdx(selected_vars, cat_vars)

    full_x[num_vars] = full_x[num_vars].astype("float32")
    full_x[cat_vars] = full_x[cat_vars].astype("int32")
    
    scaler = MinMaxScaler()
    full_x = scaler.fit_transform(full_x)
    print(full_x.shape)
    
    # === Preprocessing (Test) ===
    test_x = df_test[df_test["product_code"].isin(line)]
    test_x["line"] = lbe.transform(test_x["line"])
    test_x = test_x[selected_vars]
    test_x = test_x.fillna(-1.0)
    
    test_x[num_vars] = test_x[num_vars].astype("float32")
    test_x[cat_vars] = test_x[cat_vars].astype("int32")
    test_x = scaler.transform(test_x)
#     break
    
    # === Neural Architecture Search with Optuna & Training ===
    line_full_pred = np.zeros((len(full_x), 3))
    line_test_pred = np.zeros((len(test_x), 3))
    kfolds_spliter = StratifiedKFold(3 if CFG.debug else 5, shuffle=True, random_state=42)
    for fold, (train_idx, valid_idx) in enumerate(kfolds_spliter.split(full_x, full_y_cls)):

        train_x = full_x[train_idx]
        train_y = full_y[train_idx]
        valid_x = full_x[valid_idx]
        valid_y = full_y[valid_idx]

        output_container = {"model": None, "optuna_score": np.inf}
        optuna_direction = "minimize"
        optuna_trials = 3 if CFG.debug else 100
        optuna_earlyStopping = Optuna_EarlyStoppingCallback(max(5, 100), direction=optuna_direction)
        optuna_timout = 6 * 3600
        optuna_study = create_study(direction=optuna_direction, sampler=TPESampler(seed=GLOBAL_SEED))

        optuna_study.optimize(
            lambda trial: optuna_objective_function(
                trial, GLOBAL_SEED,
                train_x, train_y, valid_x, valid_y, output_container=output_container,
            ),
            n_jobs=1, n_trials=optuna_trials, timeout=optuna_timout, callbacks=[optuna_earlyStopping]
        )

        model = output_container["model"]
        
        tmp_pred = []
        dl = DataLoader(CustomDataset(torch.from_numpy(valid_x).to(torch.float32), torch.zeros(len(valid_x)).to(torch.float32)), shuffle=False, batch_size=CFG.batch_size)
        for batch in dl:
            with torch.no_grad():
                tmp_pred.append(model(batch["feature"]).detach().cpu().numpy())
        tmp_pred = np.concatenate(tmp_pred, axis=0, dtype="float32")
        line_full_pred[valid_idx] = softmax(tmp_pred, axis=-1)
        
        tmp_pred = []
        dl = DataLoader(CustomDataset(torch.from_numpy(test_x).to(torch.float32), torch.zeros(len(test_x)).to(torch.float32)), shuffle=False, batch_size=CFG.batch_size)
        for batch in dl:
            with torch.no_grad():
                tmp_pred.append(model(batch["feature"]).detach().cpu().numpy())
        tmp_pred = np.concatenate(tmp_pred, axis=0, dtype="float32")
        line_test_pred[:] += softmax(tmp_pred, axis=-1) / kfolds_spliter.n_splits
        
    # Evaluation
    y_true = full_y_cls.copy()
    y_pred = line_full_pred.copy()

    # Transform quality to class
    y_true_class = y_true
    y_pred_class = y_pred.argmax(axis=1)
    eval_acc = accuracy_score(y_true_class, y_pred_class)
    eval_f1 = f1_score(y_true_class, y_pred_class, average="macro")
    
    # Save values
    line_score["_".join(line)]["valid"]["mae"] = eval_mae
    line_score["_".join(line)]["valid"]["r2"] = eval_r2
    line_score["_".join(line)]["valid"]["accuracy"] = eval_acc
    line_score["_".join(line)]["valid"]["f1"] = eval_f1
    valid_pred["_".join(line)] = y_pred
    
    # Inference
    y_pred = line_test_pred.copy()
    
    # Save values
    test_pred["_".join(line)] = y_pred

df_meta["cls_cls"] = [valid_pred, test_pred]

[I 2023-02-17 16:09:13,256] A new study created in memory with name: no-name-5d558881-f0fd-4b80-8bec-337c5ed16ccb


(249, 1173)


[I 2023-02-17 16:09:23,601] Trial 0 finished with value: 1.0549782101924603 and parameters: {'dropoutRate': 0.42472407130841755, 'hidden_layers': 10, 'layer_depth': 2, 'nblocks': 7, 'batch_norm': True}. Best is trial 0 with value: 1.0549782101924603.
[I 2023-02-17 16:09:30,137] Trial 1 finished with value: 1.1013881609990046 and parameters: {'dropoutRate': 0.2935967122017216, 'hidden_layers': 2, 'layer_depth': 2, 'nblocks': 8, 'batch_norm': False}. Best is trial 0 with value: 1.0549782101924603.
[I 2023-02-17 16:09:39,990] Trial 2 finished with value: 1.0611988122646625 and parameters: {'dropoutRate': 0.21235069657748148, 'hidden_layers': 10, 'layer_depth': 2, 'nblocks': 7, 'batch_norm': True}. Best is trial 0 with value: 1.0549782101924603.
[I 2023-02-17 16:09:48,338] Trial 3 finished with value: 1.1222480810605562 and parameters: {'dropoutRate': 0.3100427059120603, 'hidden_layers': 4, 'layer_depth': 2, 'nblocks': 5, 'batch_norm': True}. Best is trial 0 with value: 1.0549782101924603.

[I 2023-02-17 16:13:20,346] Trial 32 finished with value: 1.0412365885881276 and parameters: {'dropoutRate': 0.3329442813509002, 'hidden_layers': 7, 'layer_depth': 2, 'nblocks': 8, 'batch_norm': True}. Best is trial 30 with value: 1.034920600744394.
[I 2023-02-17 16:13:33,427] Trial 33 finished with value: 1.0629781117806067 and parameters: {'dropoutRate': 0.32790548815666465, 'hidden_layers': 6, 'layer_depth': 2, 'nblocks': 8, 'batch_norm': True}. Best is trial 30 with value: 1.034920600744394.
[I 2023-02-17 16:13:44,310] Trial 34 finished with value: 1.0947292676338782 and parameters: {'dropoutRate': 0.2864510506020812, 'hidden_layers': 8, 'layer_depth': 2, 'nblocks': 7, 'batch_norm': True}. Best is trial 30 with value: 1.034920600744394.
[I 2023-02-17 16:13:57,267] Trial 35 finished with value: 1.0507301688194275 and parameters: {'dropoutRate': 0.36241360022443386, 'hidden_layers': 7, 'layer_depth': 2, 'nblocks': 8, 'batch_norm': True}. Best is trial 30 with value: 1.034920600744394

[I 2023-02-17 16:19:21,954] Trial 64 finished with value: 1.0652072154558623 and parameters: {'dropoutRate': 0.4110439178494267, 'hidden_layers': 6, 'layer_depth': 2, 'nblocks': 8, 'batch_norm': True}. Best is trial 30 with value: 1.034920600744394.
[I 2023-02-17 16:19:33,285] Trial 65 finished with value: 1.0581440925598145 and parameters: {'dropoutRate': 0.32097771168218237, 'hidden_layers': 7, 'layer_depth': 2, 'nblocks': 7, 'batch_norm': True}. Best is trial 30 with value: 1.034920600744394.
[I 2023-02-17 16:19:46,104] Trial 66 finished with value: 1.0625953811865587 and parameters: {'dropoutRate': 0.35192836373963, 'hidden_layers': 8, 'layer_depth': 2, 'nblocks': 8, 'batch_norm': True}. Best is trial 30 with value: 1.034920600744394.
[I 2023-02-17 16:19:57,518] Trial 67 finished with value: 1.0656416370318487 and parameters: {'dropoutRate': 0.43814130628434805, 'hidden_layers': 5, 'layer_depth': 2, 'nblocks': 7, 'batch_norm': True}. Best is trial 30 with value: 1.034920600744394.


...

[I 2023-02-17 17:53:04,383] Trial 75 finished with value: 0.7181340124871995 and parameters: {'dropoutRate': 0.29353332224482437, 'hidden_layers': 5, 'layer_depth': 2, 'nblocks': 8, 'batch_norm': False}. Best is trial 6 with value: 0.7154257165061103.
[I 2023-02-17 17:53:11,610] Trial 76 finished with value: 0.7172407938374413 and parameters: {'dropoutRate': 0.24017793804059076, 'hidden_layers': 4, 'layer_depth': 2, 'nblocks': 8, 'batch_norm': False}. Best is trial 6 with value: 0.7154257165061103.
[I 2023-02-17 17:53:20,042] Trial 77 finished with value: 0.7181247042285072 and parameters: {'dropoutRate': 0.3055896160312537, 'hidden_layers': 4, 'layer_depth': 2, 'nblocks': 7, 'batch_norm': False}. Best is trial 6 with value: 0.7154257165061103.
[I 2023-02-17 17:53:28,510] Trial 78 finished with value: 0.7349666158358256 and parameters: {'dropoutRate': 0.27350498936410045, 'hidden_layers': 7, 'layer_depth': 2, 'nblocks': 7, 'batch_norm': False}. Best is trial 6 with value: 0.71542571650

[I 2023-02-17 17:57:32,279] Trial 6 finished with value: 0.7640281584527757 and parameters: {'dropoutRate': 0.5645269111408631, 'hidden_layers': 3, 'layer_depth': 2, 'nblocks': 2, 'batch_norm': False}. Best is trial 1 with value: 0.7597781750890944.
[I 2023-02-17 17:57:36,512] Trial 7 finished with value: 0.8150349424944984 and parameters: {'dropoutRate': 0.6850384088698768, 'hidden_layers': 4, 'layer_depth': 2, 'nblocks': 2, 'batch_norm': True}. Best is trial 1 with value: 0.7597781750890944.
[I 2023-02-17 17:57:41,949] Trial 8 finished with value: 0.7954005698362986 and parameters: {'dropoutRate': 0.27322294090686733, 'hidden_layers': 6, 'layer_depth': 2, 'nblocks': 2, 'batch_norm': True}. Best is trial 1 with value: 0.7597781750890944.
[I 2023-02-17 17:57:53,631] Trial 9 finished with value: 0.7829590605364906 and parameters: {'dropoutRate': 0.5975133706123892, 'hidden_layers': 4, 'layer_depth': 2, 'nblocks': 5, 'batch_norm': True}. Best is trial 1 with value: 0.7597781750890944.
[I

[I 2023-02-17 18:02:02,719] Trial 37 finished with value: 0.7597808871004317 and parameters: {'dropoutRate': 0.23690392569231739, 'hidden_layers': 8, 'layer_depth': 2, 'nblocks': 7, 'batch_norm': False}. Best is trial 15 with value: 0.7597779830296835.
[I 2023-02-17 18:02:13,681] Trial 38 finished with value: 0.7601990964677598 and parameters: {'dropoutRate': 0.5441904266988772, 'hidden_layers': 9, 'layer_depth': 2, 'nblocks': 8, 'batch_norm': False}. Best is trial 15 with value: 0.7597779830296835.
[I 2023-02-17 18:02:32,737] Trial 39 finished with value: 0.8272875779204898 and parameters: {'dropoutRate': 0.30710669645154076, 'hidden_layers': 10, 'layer_depth': 2, 'nblocks': 7, 'batch_norm': True}. Best is trial 15 with value: 0.7597779830296835.
[I 2023-02-17 18:02:40,390] Trial 40 finished with value: 0.7617867257859972 and parameters: {'dropoutRate': 0.6344384675108197, 'hidden_layers': 9, 'layer_depth': 2, 'nblocks': 6, 'batch_norm': False}. Best is trial 15 with value: 0.75977798

[I 2023-02-17 18:07:18,442] Trial 68 finished with value: 0.7597779764069451 and parameters: {'dropoutRate': 0.5473368659195964, 'hidden_layers': 2, 'layer_depth': 2, 'nblocks': 8, 'batch_norm': False}. Best is trial 67 with value: 0.7597779697842069.
[I 2023-02-17 18:07:26,976] Trial 69 finished with value: 0.759781175189548 and parameters: {'dropoutRate': 0.6569058112467019, 'hidden_layers': 2, 'layer_depth': 2, 'nblocks': 7, 'batch_norm': False}. Best is trial 67 with value: 0.7597779697842069.
[I 2023-02-17 18:07:33,329] Trial 70 finished with value: 0.7636312577459548 and parameters: {'dropoutRate': 0.6543054221576965, 'hidden_layers': 2, 'layer_depth': 2, 'nblocks': 5, 'batch_norm': False}. Best is trial 67 with value: 0.7597779697842069.
[I 2023-02-17 18:07:42,063] Trial 71 finished with value: 0.7597779697842069 and parameters: {'dropoutRate': 0.5859117352153185, 'hidden_layers': 3, 'layer_depth': 2, 'nblocks': 7, 'batch_norm': False}. Best is trial 67 with value: 0.75977796978

[I 2023-02-17 18:12:04,133] Trial 99 finished with value: 0.7597779764069451 and parameters: {'dropoutRate': 0.7376540924773095, 'hidden_layers': 3, 'layer_depth': 2, 'nblocks': 8, 'batch_norm': False}. Best is trial 73 with value: 0.7597779598500993.


## Meta Learning (no class weight,  normalizing)

In [47]:
df_meta_backup = df_meta.copy()

In [48]:
output_prob_valid = {}
output_prob_test = {}

for line in line_split:
    full_x = []
    test_x = []
    for k, v in df_meta.items():
        full_x.append(v[0]["_".join(line)])
        test_x.append(v[1]["_".join(line)])
    full_x = np.concatenate([i.reshape(-1, 1) if len(i.shape) == 1 else i for i in full_x], axis=1)
    full_x[: ,0] = (full_x[: ,0] - 0.5) * 10
    full_x[: ,4] = (full_x[: ,4]) / 3
    full_y = df_full.loc[df_full["product_code"].isin(line), "y_class"].values
    test_x = np.concatenate([i.reshape(-1, 1) if len(i.shape) == 1 else i for i in test_x], axis=1)
    test_x[: ,0] = (test_x[: ,0] - 0.5) * 10
    test_x[: ,4] = (test_x[: ,4]) / 3

    meta_learner = lm.LogisticRegression(multi_class="multinomial", penalty='elasticnet', solver="saga", l1_ratio=0.5, class_weight=None, random_state=42)
    meta_learner.fit(full_x, full_y)
    output_prob_valid["_".join(line)] = meta_learner.predict_proba(full_x)
    output_prob_test["_".join(line)] = meta_learner.predict_proba(test_x)
    
df_meta["meta_learning"] = [output_prob_valid, output_prob_test]

## Threshold Optimization

### Create infernced value table for analysis

In [49]:
raw_output_container_train = []
raw_output_container_test = []

for line in line_split:
    train_x = []
    test_x = []
    for k, v in df_meta.items():
        train_x.append(v[0]["_".join(line)])
        test_x.append(v[1]["_".join(line)])
    train_x = np.concatenate([i.reshape(-1, 1) if len(i.shape) == 1 else i for i in train_x], axis=1)
    train_y = df_full.loc[df_full["product_code"].isin(line), "y_class"].values
    test_x = np.concatenate([i.reshape(-1, 1) if len(i.shape) == 1 else i for i in test_x], axis=1)
    
    raw_output = pd.DataFrame(train_x, columns=["reg_qual", "cls_cls0", "cls_cls1", "cls_cls2", "reg_cls", "meta_learning_cls0", "meta_learning_cls1", "meta_learning_cls2"])
    raw_output["y_true"] = train_y
    raw_output["product_code"] = df_full.loc[df_full["product_code"].isin(line), "product_code"].values
    raw_output_container_train.append(raw_output)

    raw_output = pd.DataFrame(test_x, columns=["reg_qual", "cls_cls0", "cls_cls1", "cls_cls2", "reg_cls", "meta_learning_cls0", "meta_learning_cls1", "meta_learning_cls2"])
    raw_output["product_code"] = df_test.loc[df_test["product_code"].isin(line), "product_code"].values
    raw_output["product_id"] = df_test.loc[df_test["product_code"].isin(line), "product_id"].values
    raw_output_container_test.append(raw_output)

In [50]:
raw_output_container_train = pd.concat(raw_output_container_train)
raw_output_container_train.head()

,reg_qual,cls_cls0,cls_cls1,cls_cls2,reg_cls,meta_learning_cls0,meta_learning_cls1,meta_learning_cls2,y_true,product_code
0,0.523109,0.103706,0.736767,0.159528,0.749796,0.262618,0.551548,0.185834,0,A_31
1,0.533842,0.009290,0.840490,0.150219,0.937185,0.180254,0.586345,0.233400,1,A_31
2,0.525907,0.421447,0.263269,0.315283,0.685504,0.502267,0.299631,0.198102,2,A_31
3,0.531008,0.000904,0.770070,0.229026,1.174997,0.154770,0.550802,0.294428,1,A_31
4,0.533135,0.413459,0.378453,0.208087,1.230163,0.313804,0.335700,0.350497,1,A_31


In [51]:
with pd.ExcelWriter(architecture_path + "train_threshold_analysis.xlsx") as writer:
    raw_output_container_train.groupby("y_true").describe().T.reset_index().to_excel(writer, sheet_name="all")
    for line in line_split:
        df_tmp = raw_output_container_train[raw_output_container_train["product_code"].isin(line)].groupby("y_true").describe().T.reset_index()
        df_tmp.to_excel(writer, sheet_name="_".join(line))

In [52]:
raw_output_container_test = pd.concat(raw_output_container_test)
raw_output_container_test.head()

,reg_qual,cls_cls0,cls_cls1,cls_cls2,reg_cls,meta_learning_cls0,meta_learning_cls1,meta_learning_cls2,product_code,product_id
0,0.532698,0.064713,0.773664,0.161623,1.003089,0.194413,0.551085,0.254502,A_31,TEST_003
1,0.535916,0.065366,0.761725,0.172908,1.227721,0.159116,0.522497,0.318387,A_31,TEST_004
2,0.534955,0.064480,0.767025,0.168495,1.200589,0.162866,0.529277,0.307857,A_31,TEST_005
3,0.530895,0.063700,0.779450,0.156850,0.987232,0.196798,0.557579,0.245623,A_31,TEST_006
4,0.527383,0.365304,0.483014,0.151682,0.664068,0.432551,0.383507,0.183942,A_31,TEST_007


In [53]:
df_tmp = df_test[["product_id", "product_code"]]
for line in line_split:
    df_tmp.loc[df_tmp["product_code"].isin(line).values, raw_output_container_test.columns[:-1]] = raw_output_container_test[raw_output_container_test["product_code"].isin(line).values].iloc[:, :-1].values

## Inference with Threshold

In [150]:
# # 주혁님 threshold 계산 algorithm
# submit = pd.read_csv('/content/모델4개2.csv', encoding = 'cp949')
# submit['0.748 결과'][(submit['0.748 결과'] != 0) & (submit['class0'] > 0.45) & (submit['Class를 회귀로(모델3개)'] <= 0.75) & (submit['Class를 회귀로(모델1개)'] <= 0.75)] = 0
# s = pd.read_csv('/content/drive/MyDrive/LG_Aimers2/open (7)/sample_submission.csv')
# s['Y_Class'] = submit['0.748 결과']
# s.to_csv('SotaToCha_0.csv',index=False)

In [151]:
def get_threshold_params(x, norm_params={"reg_qual": 0.05, "reg_cls": 0.5, "cls_cls": 0.05, "meta_learning": 0.05}):
    '''
        reg_qual : percentage of reduction on upper and lower bound for class 1
        cls_reg : multiplier of standard deviation on upper and lower bound for class 1 (mean + std * alpha, mean - std * alpha)
        cls_cls : minimum probability for class 1
        meta_learning : minimum probability for class 1 (same as cls_cls)
    '''

    threshold_dic = {"reg_qual": {}, "reg_cls": {}, "cls_cls": {}, "meta_learning": {}}

    df_tmp = x.copy()
    df_tmp["tmp"] = df_tmp["product_code"].apply(lambda x: "A_31" if x == "A_31" else "T_31_O_31")

    # reg_qual
    df_stats = df_tmp.groupby(["tmp", "y_class"])["y_quality"].describe().T
    for k in ["A_31", "T_31_O_31"]:
        tmp_range = (df_stats[k].loc["max", 1] - df_stats[k].loc["min", 1])
        threshold_dic["reg_qual"][k] = [df_stats[k].loc["min", 1] + tmp_range * norm_params["reg_qual"], df_stats[k].loc["max", 1] - tmp_range * norm_params["reg_qual"]]

    # reg_cls
    df_stats = df_tmp.groupby("tmp")["y_class"].describe().T
    for k in ["A_31", "T_31_O_31"]:
        threshold_dic["reg_cls"][k] = [df_stats.loc["mean", k] - df_stats.loc["std", k] * norm_params["reg_cls"], df_stats.loc["mean", k] + df_stats.loc["std", k] * norm_params["reg_cls"]]

    # cls_cls
    df_stats = df_tmp.groupby("tmp")["y_class"].value_counts(normalize=True)
    for k in ["A_31", "T_31_O_31"]:
        threshold_dic["cls_cls"][k] = [df_stats[(k, 1)] * (1 + norm_params["cls_cls"]), 1]

    # meta_learning
    df_stats = df_tmp.groupby("tmp")["y_class"].value_counts(normalize=True)
    for k in ["A_31", "T_31_O_31"]:
        threshold_dic["meta_learning"][k] = [df_stats[(k, 1)] * (1 + norm_params["meta_learning"]), 1]

    for k, v in threshold_dic.items():
        print(k)
        print(v)

    return threshold_dic

In [152]:
def get_inference_label(x, class_spliter):
    df_tmp = x.copy()
    df_tmp["tmp"] = df_tmp["product_code"].apply(lambda x: "A_31" if x == "A_31" else "T_31_O_31")

    # reg_qual
    df_tmp["label_reg_qual"] = 1
    for k, v in class_spliter["reg_qual"].items():
        tmp_labeld = []
        for i in df_tmp.loc[df_tmp["tmp"] == k, "reg_qual"].values:
            if i < v[0]:
                tmp_labeld.append(0)
            elif i > v[1]:
                tmp_labeld.append(2)
            else:
                tmp_labeld.append(1)
        df_tmp.loc[df_tmp["tmp"] == k, "label_reg_qual"] = tmp_labeld

    # reg_cls
    df_tmp["label_reg_cls"] = 1
    for k, v in class_spliter["reg_cls"].items():
        tmp_labeld = []
        for i in df_tmp.loc[df_tmp["tmp"] == k, "reg_cls"].values:
            if i < v[0]:
                tmp_labeld.append(0)
            elif i > v[1]:
                tmp_labeld.append(2)
            else:
                tmp_labeld.append(1)
        df_tmp.loc[df_tmp["tmp"] == k, "label_reg_cls"] = tmp_labeld

    # cls_cls
    df_tmp["label_cls_cls"] = 1
    for k, v in class_spliter["cls_cls"].items():
        tmp_labeld = []
        for i in df_tmp.loc[df_tmp["tmp"] == k, ["cls_cls0", "cls_cls1", "cls_cls2"]].values:
            if (i[1] >= v[0]) & (i[1] <= v[1]):
                tmp_labeld.append(1)
            else:
                tmp_labeld.append(0 if np.argmax([i[0], i[2]]) == 0 else 2)
        df_tmp.loc[df_tmp["tmp"] == k, "label_cls_cls"] = tmp_labeld

    # meta_learning
    df_tmp["label_meta_learning"] = 1
    for k, v in class_spliter["meta_learning"].items():
        tmp_labeld = []
        for i in df_tmp.loc[df_tmp["tmp"] == k, ["meta_learning_cls0", "meta_learning_cls1", "meta_learning_cls2"]].values:
            if (i[1] >= v[0]) & (i[1] <= v[1]):
                tmp_labeld.append(1)
            else:
                tmp_labeld.append(0 if np.argmax([i[0], i[2]]) == 0 else 2)
        df_tmp.loc[df_tmp["tmp"] == k, "label_meta_learning"] = tmp_labeld
    
    # ensemble prediction
    df_tmp["majority_vote"] = df_tmp.filter(regex="label_*").apply(lambda x: Counter(x).most_common(1)[0][0], axis=1)
    
    return df_tmp

In [153]:
df_full["y_class"].value_counts(normalize=True)

1    0.680602
2    0.172241
0    0.147157
Name: y_class, dtype: float64

In [176]:
params = {"reg_qual": 0.25, "reg_cls": 0.3, "cls_cls": 0.05, "meta_learning": 0.0}
class_spliter = get_threshold_params(df_full, params)

reg_qual
{'A_31': [0.52752499975, 0.53240357125], 'T_31_O_31': [0.5276186504999999, 0.5324305555]}
reg_cls
{'A_31': [0.8265440529880919, 1.2618093606665268], 'T_31_O_31': [0.8869406978981454, 1.1359819382050065]}
cls_cls
{'A_31': [0.49759036144578317, 1], 'T_31_O_31': [0.8694842406876792, 1]}
meta_learning
{'A_31': [0.4738955823293173, 1], 'T_31_O_31': [0.828080229226361, 1]}


In [167]:
train_inference = get_inference_label(raw_output_container_train, class_spliter)
for i in ["label_reg_qual", "label_reg_cls", "label_cls_cls", "label_meta_learning", "majority_vote"]:
    print(f"=== {i} ===")
    print("value counts")
    display(train_inference[i].value_counts())
    display(train_inference[i].value_counts(normalize=True))
    display(train_inference.groupby("product_code")[i].value_counts(normalize=True))
    print("\n")
#     print("acc:", accuracy_score(train_inference["y_true"], test_inference[i]))
#     print("f1:", f1_score(train_inference["y_true"], train_inference[i], average="macro"))

=== label_reg_qual ===
value counts


1    376
2    121
0    101
Name: label_reg_qual, dtype: int64

1    0.628763
2    0.202341
0    0.168896
Name: label_reg_qual, dtype: float64

product_code  label_reg_qual
A_31          2                 0.485944
              1                 0.349398
              0                 0.164659
O_31          1                 0.833333
              0                 0.166667
T_31          1                 0.827988
              0                 0.172012
Name: label_reg_qual, dtype: float64



=== label_reg_cls ===
value counts


...

product_code  label_meta_learning
A_31          1                      0.630522
              0                      0.184739
              2                      0.184739
O_31          1                      1.000000
T_31          1                      0.629738
              2                      0.370262
Name: label_meta_learning, dtype: float64



=== majority_vote ===
value counts


1    479
2     65
0     54
Name: majority_vote, dtype: int64

1    0.801003
2    0.108696
0    0.090301
Name: majority_vote, dtype: float64

product_code  majority_vote
A_31          1                0.558233
              2                0.261044
              0                0.180723
O_31          1                1.000000
T_31          1                0.973761
              0                0.026239
Name: majority_vote, dtype: float64

In [168]:
for i in ["label_reg_qual", "label_reg_cls", "label_cls_cls", "majority_vote"]:
    print(f"=== {i} ===")
    print("acc:", accuracy_score(train_inference["y_true"], train_inference[i]))
    print("f1:", f1_score(train_inference["y_true"], train_inference[i], average="macro"))

=== label_reg_qual ===
acc: 0.5986622073578596
f1: 0.4720435747734599
=== label_reg_cls ===
acc: 0.6822742474916388
f1: 0.5383801178451509
=== label_cls_cls ===
acc: 0.7073578595317725
f1: 0.44750692930971714
=== majority_vote ===
acc: 0.7073578595317725
f1: 0.5365556771781933


In [177]:
test_inference = get_inference_label(raw_output_container_test, class_spliter)
for i in ["label_reg_qual", "label_reg_cls", "label_cls_cls", "label_meta_learning", "majority_vote"]:
    print(f"=== {i} ===")
    print("value counts")
    display(test_inference[i].value_counts())
    display(test_inference[i].value_counts(normalize=True))
    display(test_inference.groupby("product_code")[i].value_counts(normalize=True))
    print("\n")
#     print("acc:", accuracy_score(train_inference["y_true"], test_inference[i]))
#     print("f1:", f1_score(train_inference["y_true"], train_inference[i], average="macro"))

=== label_reg_qual ===
value counts


1    286
2     18
0      6
Name: label_reg_qual, dtype: int64

1    0.922581
2    0.058065
0    0.019355
Name: label_reg_qual, dtype: float64

product_code  label_reg_qual
A_31          1                 0.641791
              2                 0.268657
              0                 0.089552
O_31          1                 1.000000
T_31          1                 1.000000
Name: label_reg_qual, dtype: float64



=== label_reg_cls ===
value counts


...

product_code  label_meta_learning
A_31          0                      0.582090
              1                      0.417910
O_31          1                      1.000000
T_31          1                      0.941423
              2                      0.058577
Name: label_meta_learning, dtype: float64



=== majority_vote ===
value counts


1    268
0     39
2      3
Name: majority_vote, dtype: int64

1    0.864516
0    0.125806
2    0.009677
Name: majority_vote, dtype: float64

product_code  majority_vote
A_31          0                0.582090
              1                0.373134
              2                0.044776
O_31          1                1.000000
T_31          1                1.000000
Name: majority_vote, dtype: float64

In [158]:
test_inference.sort_values("product_id").to_csv(architecture_path + "test_rawoutput.csv", index=False)

## Submission

In [159]:
inference_col = "majority_vote"

In [160]:
df_test_tmp = df_test[["product_id", "line"]]
df_test_tmp["target_class"] = 1

for line in line_split:
    df_test_tmp.loc[df_tmp["product_code"].isin(line), "target_class"] = test_inference.loc[test_inference["product_code"].isin(line), inference_col].values

df_test_tmp["target_class"] = df_test_tmp["target_class"].astype("int32")

In [161]:
df_test_tmp

,product_id,line,target_class
0,TEST_000,T100306,1
1,TEST_001,T100304,1
2,TEST_002,T100304,1
3,TEST_003,T010305,1
4,TEST_004,T010306,1
...,...,...,...
305,TEST_305,T100306,1
306,TEST_306,T100304,1
307,TEST_307,T100306,1
308,TEST_308,T100306,1


In [162]:
submission = pd.read_csv("../datasets/sample_submission.csv")
submission["Y_Class"] = df_test_tmp["target_class"].values

In [163]:
submission

,PRODUCT_ID,Y_Class
0,TEST_000,1
1,TEST_001,1
2,TEST_002,1
3,TEST_003,1
4,TEST_004,1
...,...,...
305,TEST_305,1
306,TEST_306,1
307,TEST_307,1
308,TEST_308,1


In [164]:
submission.to_csv(architecture_path + "submission_" + architecture_name + ".csv", index=False)